In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

In [5]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(5, 110, 5)
non_lin = [2]
non_lin_dict = {1: "sigmoid", 2: "relu"}
alpha = 0

#empty dictionary
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c])) 
        
        y_train[y_train < 0] = 0
        y_test[y_test < 0 ] = 0
        
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, lr = 1e-4)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 50)
        

        
        w, o, s  = model.get_params()
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                 100%
[                     ]

testing non-linearity:  relu ... testing  5  pc's...
new best model for cell  517442753 . r2 test:  0.0456279314541
new best model for cell  517444738 . r2 test:  -0.0228808608717
new best model for cell  517443331 . r2 test:  0.0103987164378
new best model for cell  517442436 . r2 test:  0.00777864870901
new best model for cell  517445575 . r2 test:  -0.0295696995782
new best model for cell  517444426 . r2 test:  0.0218410128247
new best model for cell  517442471 . r2 test:  -0.000850413243955
new best model for cell  517443181 . r2 test:  -0.0145824422996
new best model for cell  517443087 . r2 test:  0.0252485094953
new best model for cell  517446833 . r2 test:  -0.0223882541295
new best model for cell  517443827 . r2 test:  -0.0331371097071
new best model for cell  517443924 . r2 test:  -0.0147612362041
new best model for cell  517443957 . r2 test:  0.0497759127745
new best model for cell  517442722 . r2 test:  0.0543706833585
new best model for cell  517446488 . r2 test:  0.024689

[#                    ] | ETA: 00:12:30

new best model for cell  517444738 . r2 test:  -0.0159637385738
new best model for cell  517445575 . r2 test:  -0.00221954297692
new best model for cell  517442471 . r2 test:  0.0265663166667
new best model for cell  517443087 . r2 test:  0.0290131240619
new best model for cell  517446833 . r2 test:  -0.016761203719
new best model for cell  517444175 . r2 test:  0.0123100924842
new best model for cell  517443484 . r2 test:  -0.0215801413244
new best model for cell  517443571 . r2 test:  0.00789291283546
testing non-linearity:  relu ... testing  15  pc's...


[###                  ] | ETA: 00:21:13

...

[###################  ] | ETA: 00:16:39

testing non-linearity:  relu ... testing  100  pc's...


[#################### ] | ETA: 00:08:59

new best model for cell  517444738 . r2 test:  0.0652277034263
new best model for cell  517442436 . r2 test:  0.0679360423045
testing non-linearity:  relu ... testing  105  pc's...


[#####################] | ETA: 00:00:00
Total time elapsed: 03:25:22


In [8]:
pkl_file = open('./boc/formatted/LNE_relu', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()